In [1]:
# load
import pandas as pd
import lightgbm

X_train_in = pd.read_csv("X_train.csv", index_col=0)
y_train_in = pd.read_csv("y_train.csv", index_col=0)
X_validation = pd.read_csv("X_test.csv", index_col=0)

stud_info = pd.read_csv("studs_info.csv", index_col=0)

In [2]:
# rename columns
field_map = {
    "STD_ID": "stud",
    "НАПРАВЛЕНИЕ": "profession",
    "ГОД": "year",
    "АТТЕСТАЦИЯ": "exam_type",
    "ДИСЦИПЛИНА": "discipline",
    "КУРС": "course",
    "СЕМЕСТР": "semester",
}

for field_name, new_field_name in field_map.items():
    X_train_in[new_field_name] = X_train_in.pop(field_name)
    X_validation[new_field_name] = X_validation.pop(field_name)

In [3]:
stud_info

,number,Пол,Статус,Дата выпуска,Категория обучения,Форма обучения,Шифр,направление (специальность),,Образование,Дата выдачи,Что именно закончил
STD_ID,,,,,,,,,,,,
99136,13951.0,Ж,ВЫП,2019-08-31,БП,Д,31.05.01,Лечебное дело,2629.0,СР,2013-06-26 00:00:00,ИНОГОРОДНЯЯ СРЕДНЯЯ ШКОЛА
99112,13998.0,Ж,ВЫП,2019-08-31,БП,Д,31.05.01,Лечебное дело,8240.0,СР,2012-06-23 00:00:00,"МОУ СОШ N 2, ПЕСТОВО"
99109,13999.0,Ж,ВЫП,2019-08-31,БП,Д,31.05.01,Лечебное дело,2314.0,СР,2013-06-25 00:00:00,ИНОГОРОДНЯЯ СРЕДНЯЯ ШКОЛА
99125,13953.0,Ж,ВЫП,2019-08-31,БП,Д,31.05.01,Лечебное дело,2345.0,СР,2013-06-23 00:00:00,"Гимназия ""ГАРМОНИЯ"""
100676,13950.0,М,ВЫП,2019-08-31,БП,Д,31.05.01,Лечебное дело,2650.0,СР,2012-06-29 00:00:00,ИНОГОРОДНЯЯ СРЕДНЯЯ ШКОЛА
...,...,...,...,...,...,...,...,...,...,...,...,...
100678,978.0,Ж,СТ,NaN,БП,Д,31.05.01,Лечебное дело,2388.0,СР,2013-06-21 00:00:00,ИНОГОРОДНЯЯ СРЕДНЯЯ ШКОЛА
100663,1021.0,Ж,СТ,NaN,ДОГ,Д,31.05.01,Лечебное дело,2554.0,СР,2013-06-27 00:00:00,ИНОГОРОДНЯЯ СРЕДНЯЯ ШКОЛА
191630,6447.0,Ж,СТ,NaN,ДОГ,Д,31.05.01,Лечебное дело,6692.0,СР,2014-05-31 00:00:00,ИНОГОРОДНЯЯ СРЕДНЯЯ ШКОЛА


In [4]:
X_train_in

,stud,profession,year,exam_type,discipline,course,semester
51081,175711,38.03.02 Менеджмент,2018-2019,Экзамен,Маркетинг,2.0,4.0
72921,100647,31.05.01 Лечебное дело,2015-2016,Экзамен,"Пропедевтика внутренних болезней, лучевая диаг...",3.0,5.0
96438,199311,31.05.03 Стоматология,2019-2020,Экзамен,Химия,1.0,1.0
273044,162002,38.03.02 Менеджмент,2017-2018,Экзамен,Статистика,2.0,3.0
371309,108384,31.05.01 Лечебное дело,2018-2019,Экзамен,Педиатрия,5.0,9.0
...,...,...,...,...,...,...,...
224856,176141,40.03.01 Юриспруденция,2017-2018,Курсовая работа,Конституционное право,1.0,2.0
260998,182794,31.05.01 Лечебное дело,2018-2019,Экзамен,Нормальная физиология,2.0,4.0
57245,162391,31.05.01 Лечебное дело,2016-2017,Дифференцированный зачет,Иностранный язык,1.0,2.0
218529,202479,38.03.01 Экономика,2017-2018,Дифференцированный зачет,Информатика,1.0,1.0


In [5]:
# encode labels
from sklearn import preprocessing
fields = ["profession", "year", "exam_type", "discipline"]
le_s = [preprocessing.LabelEncoder() for _ in fields]

for field_name, le in zip(fields, le_s):
    le.fit(pd.concat([X_train_in[field_name], X_validation[field_name]]))

    X_train_in[field_name] = le.transform(X_train_in[field_name])
    X_validation[field_name] = le.transform(X_validation[field_name])

# 69.0 to 69
for field_name in ("course", "semester"):
    X_train_in[field_name] = X_train_in[field_name].astype(int)
    X_validation[field_name] = X_validation[field_name].astype(int)

# add marks
X_train_in["mark"] = y_train_in["mark"]

In [6]:
X_train_in

,stud,profession,year,exam_type,discipline,course,semester,mark
51081,175711,36,5,4,408,2,4,5
72921,100647,24,2,4,946,3,5,5
96438,199311,25,6,4,1326,1,1,5
273044,162002,36,4,4,1100,2,3,4
371309,108384,24,5,4,713,5,9,3
...,...,...,...,...,...,...,...,...
224856,176141,40,4,2,364,1,2,3
260998,182794,24,5,4,556,2,4,3
57245,162391,24,3,0,228,1,2,4
218529,202479,35,4,0,238,1,1,4


In [7]:
# means
for field_name in ("stud", "profession"):
    mean_mark = X_train_in.groupby(field_name).mean()["mark"]

    mean_name = field_name + "_mean"
    X_train_in[mean_name] = X_train_in[field_name].map(mean_mark)
    X_validation[mean_name] = X_validation[field_name].map(mean_mark)

In [8]:

from sklearn.model_selection import train_test_split
# разделение на обучающую и тестовую выборку для проверки решения
X_train, X_test, y_train, y_test = train_test_split(X_train_in, X_train_in["mark"], shuffle=True, test_size=0.2)

# выбор параметров на основе которых будет обучение
train_col = ["stud", "stud_mean", "course", "semester", "profession", "profession_mean"]

# создание и обучение модели на основе бустинга рещающих деревьев
lgb = lightgbm.LGBMRegressor(n_estimators=1000, verbose=1)

lgb.fit(X_train[train_col], y_train)


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002268 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 639
[LightGBM] [Info] Number of data points in the train set: 84046, number of used features: 6
[LightGBM] [Info] Start training from score 4.185030


LGBMRegressor(n_estimators=1000, verbose=1)

In [9]:

from sklearn.metrics import mean_absolute_error, r2_score

# вычисление метрик получившейся модели
print(mean_absolute_error(y_test, lgb.predict(X_test[train_col])))
print(r2_score(y_test, lgb.predict(X_test[train_col])))

# сохранение решения в нужном формате для отпавки
y_pred = pd.read_csv("sample_submission.csv", index_col=0)
y_pred["mark"] = lgb.predict(X_validation[train_col])

y_pred.to_csv("baseline_submission.csv")


0.4539413468183742
0.4302105995303137
